In [1]:
# load lib
import os
import tabulate
import numpy as np
from tqdm import tqdm
from det3.ops import read_pkl


In [2]:
# global variables
g_task="task3"
g_domain="domain3"
g_reuse_tag="reuse"
if g_task in ["task2", "task3"]:
    g_statistic_dict = {
        "joint_training": [f"July24-expcarlap1+-eval-jointtraining-{g_task}-{g_domain}-{g_reuse_tag}-{cv}" for cv in range(5)],
        "lwf": [f"July24-expcarlap1+-eval-lwf-{g_task}-{g_domain}-{g_reuse_tag}-{cv}" for cv in range(5)],
        "fine_tuning": [f"July24-expcarlap1+-eval-finetuning-{g_task}-{g_domain}-{g_reuse_tag}-{cv}" for cv in range(5)]
    }
elif g_task == "task1":
    g_statistic_dict = {
        "joint_training": [f"July24-expcarlap1+-eval-{g_task}-{g_domain}"],
        "lwf": [f"July24-expcarlap1+-eval-{g_task}-{g_domain}"],
        "fine_tuning": [f"July24-expcarlap1+-eval-{g_task}-{g_domain}"]
    }
else:
    raise NotImplementedError
if g_task == "task3" and g_reuse_tag=="reuse":
    g_statistic_dict = {
        "joint_training": [f"July24-expcarlap1+-eval-jointtraining-{g_task}-{g_domain}-{g_reuse_tag}-{cv}" for cv in range(5)],
        "lwf": [f"July24-expcarlap1+-eval-lwf-{g_task}-{g_domain}-{g_reuse_tag}-{cv}" for cv in range(5)],
        "fine_tuning": [f"July24-expcarlap1+-eval-finetuning-{g_task}-{g_domain}-{g_reuse_tag}-{cv}" for cv in range(2)]
    }
g_valid_classes = None
g_mAP_weights = None
g_mAPs = {}
g_cls2metric= {
    "Car": "3d@0.50",
    "Pedestrian": "3d@0.25",
    "Cyclist": "3d@0.25"
}

In [3]:
# define functions
def get_valid_classes(task:str, domain:str):
    if domain == "domain3":
        l_domain = ["Car", "Pedestrian", "Cyclist"]
    elif domain == "domain2":
        l_domain = ["Car", "Pedestrian"]
    elif domain == "domain1":
        l_domain = ["Car"]
    else:
        raise NotImplementedError
    if task == "task3":
        l_task = ["Car", "Pedestrian", "Cyclist"]
    elif task == "task2":
        l_task = ["Car", "Pedestrian"]
    elif task == "task1":
        l_task = ["Car"]
    else:
        raise NotImplementedError
    return [itm for itm in l_domain if itm in l_task]
    

def tag2pkl(tag, task:str, domain:str, cls2metric:dict):
    '''
    return result pkl according to tag
    '''
    path = f"/usr/app/incdet3/logs/July24-expcarlap1+-eval/{tag}/test_results.pkl"
    if not os.path.exists(path):
        return None
    data = read_pkl(path)
    data = data['detail']['eval.carla']['carla']
    res = {}
    for cls, v in data.items():
        res[cls] = v[cls2metric[cls]]
    return res

def domain2infopkl(domain:str):
    '''
    return the test info pkl according to domain.
    '''
    if domain == "domain3":
        path = "/usr/app/data/CARLA-P1-TASK3/CARLA_infos_test.pkl"
    elif domain == "domain2":
        path = "/usr/app/data/CARLA-P1-TASK2/CARLA_infos_test.pkl"
    elif domain == "domain1":
        path = "/usr/app/data/CARLA-P1-TASK1/CARLA_infos_test.pkl"
    else:
        raise NotImplementedError
    return read_pkl(path)
    
def compute_mAP(eval_data:dict, mAP_weights:dict):
    '''
    return the mAP:
    {
    "Car": float,
    "Pedestrian": float,
    "Cyclist": float
    }
    '''
    res = {}
    for cls, aps in eval_data.items():
        if cls not in [k for k in mAP_weights.keys()]:
            continue
        res[cls] = sum([ap * weight / sum(mAP_weights[cls])
                    for ap, weight in zip(aps, mAP_weights[cls])])
    return res

In [4]:
# compute easy, mod, hard weight for each class of the domain
# mAP_weights = {
#     "Car": [num_of_easy, num_of_mod, num_of_hard],
#     ...
# }
g_valid_classes = get_valid_classes(g_task, g_domain)
info_pkl = domain2infopkl(g_domain)
mAP_weights = {k: [0, 0, 0] for k in g_valid_classes}
for itm in tqdm(info_pkl):
    label = itm["label"]
    for obj in label.data:
        if obj.type not in g_valid_classes:
            continue
        dist = np.sqrt(obj.x **2 + obj.y **2)
        if dist <= 20:
            mAP_weights[obj.type][0] += 1
        if dist <= 30:
            mAP_weights[obj.type][1] += 1
        if dist <= 50:
            mAP_weights[obj.type][2] += 1
print(mAP_weights)
g_mAP_weights = mAP_weights

100%|██████████| 1100/1100 [00:00<00:00, 5753.79it/s]

{'Car': [1924, 2962, 5534], 'Pedestrian': [3340, 5590, 11515], 'Cyclist': [343, 657, 1542]}


In [5]:
# load each evaluation results and compute its map
for _, v in g_statistic_dict.items():
    for tag in v:
        eval_data = tag2pkl(tag, g_task, g_domain, g_cls2metric)
        if eval_data is None:
            g_mAPs[tag] = -1
            continue
        mAP = compute_mAP(eval_data, g_mAP_weights)
        g_mAPs[tag] = mAP
for k, v in g_mAPs.items():
    print(k)
    if v == -1 :
        print("evaluate result is not available.")
        continue
    for kk, vv in v.items():
        print(f"  {kk}: {vv:.1f}")

July24-expcarlap1+-eval-jointtraining-task3-domain3-reuse-0
  Car: 63.1
  Pedestrian: 67.6
  Cyclist: 51.5
July24-expcarlap1+-eval-jointtraining-task3-domain3-reuse-1
  Car: 63.2
  Pedestrian: 67.6
  Cyclist: 60.5
July24-expcarlap1+-eval-jointtraining-task3-domain3-reuse-2
  Car: 56.7
  Pedestrian: 67.6
  Cyclist: 59.9
July24-expcarlap1+-eval-jointtraining-task3-domain3-reuse-3
  Car: 65.2
  Pedestrian: 67.6
  Cyclist: 58.3
July24-expcarlap1+-eval-jointtraining-task3-domain3-reuse-4
  Car: 63.3
  Pedestrian: 67.6
  Cyclist: 58.2
July24-expcarlap1+-eval-lwf-task3-domain3-reuse-0
  Car: 65.2
  Pedestrian: 58.7
  Cyclist: 49.9
July24-expcarlap1+-eval-lwf-task3-domain3-reuse-1
  Car: 63.5
  Pedestrian: 57.1
  Cyclist: 50.0
July24-expcarlap1+-eval-lwf-task3-domain3-reuse-2
  Car: 55.4
  Pedestrian: 58.6
  Cyclist: 51.3
July24-expcarlap1+-eval-lwf-task3-domain3-reuse-3
  Car: 63.8
  Pedestrian: 58.7
  Cyclist: 50.1
July24-expcarlap1+-eval-lwf-task3-domain3-reuse-4
  Car: 64.8
  Pedestrian: 5

In [7]:
# generate a table according to the g_statistic_dict
header = ["Case"] + g_valid_classes
content = []
for statistic_tag, tags in g_statistic_dict.items():
    ctt = [statistic_tag]
    for cls in g_valid_classes:
        data = [g_mAPs[tag][cls] for tag in tags if g_mAPs[tag] != -1]
        ctt += [f"{np.mean(data):.2f}+-{np.std(data):.2f}"]
    content.append(ctt)
print(tabulate.tabulate(content, headers=header, tablefmt="pipe"))
for k, v in g_mAPs.items():
    if v == -1:
        print(f"{k} is not counted.")

| Case           | Car         | Pedestrian   | Cyclist     |
|:---------------|:------------|:-------------|:------------|
| joint_training | 62.32+-2.89 | 67.60+-0.03  | 57.68+-3.20 |
| lwf            | 62.53+-3.63 | 58.34+-0.62  | 50.38+-0.54 |
| fine_tuning    | 0.00+-0.00  | 0.00+-0.00   | 46.66+-5.30 |
